In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import ipdb
import time
import sys
import scipy.stats as stt

import chart_studio.plotly as ply
import chart_studio.tools as plytool
import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly.offline as plyoff
import plotly.subplots as plysub

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# my imports
sys.path.append('../src/')
from GA.GA import *
from GA.Objective import *
from Utils.Utils import *

In [ ]:
# make some data
np.random.seed(42)
n = 100
params = [5, 1]

data = stt.norm.rvs(loc=params[0], scale=params[1], size=n)
simName = 'Gauss(%d, %d)'%(params[0], params[1])

In [ ]:
# define distributions of interest
dists = ['NRM', 'GAM', 'LOG', 'EXP', 'CHI', 'STU', 'CAU', 'LPL', 'PAR']
results = dict.fromkeys(dists, None)

# get bounds
for dist in dists:
    # compute
    rng = PDFParamRanges(data, dist, scale=3)
    l = ComputeLikelihood(data, rng[0], dist)
    u = ComputeLikelihood(data, rng[1], dist)
    # store
    results[dist] = [rng, l, u]
    # talk
    print('%s = (%0.3f, %0.3f)'%(dist, l[0], u[0]))

In [ ]:
''' prepare GA input parameters '''
# distribution parameters
bitCnt = 16
dist = 'NRM'
lowerB = results[dist][0][0]
upperB = results[dist][0][1]
bits = [bitCnt]*len(lowerB)

# GA parameters
parmsGA = {'initPerc':0.5, 'showTopRes':10, 'populSize':100, 'numGens':100,
           'noChangeTerm':180, 'convgCrit':0.00001, 'elitism':True, 'mateType':1, 'probXover':0.8,
           'probMutate':0.3, 'probEngineer':0.2, 'optimGoal':1, 'plotFlag':True, 'printFreq':10,
           'xoverType':1, 'bits':bits, 'lowerB':lowerB, 'upperB':upperB}

# data parameters
parmsData = {'data':data, 'name':simName}

# objective parameters
parmsObj = {'function':'ComputeLikelihood',
            'arguments':{'data':None, 'params':None, 'dist':dist}}

In [ ]:
''' run the GA - hold on to your butts '''
# parameters
randSeed = None#42
verb = False
MSims = 5

# init
bestRess = [None]*MSims
bestParams = [None]*MSims
bestScores = [None]*MSims
genBests = [None]*MSims
genBestParams = [None]*MSims
genScores = [None]*MSims
randSeeds = [None]*MSims
timeStamps = [None]*MSims
figGAProgresss = [None]*MSims

for sim in range(MSims):
    print('Executing GA %d of %d'%(sim+1, MSims))
    bestRess[sim], bestParams[sim], bestScores[sim], genBests[sim],\
        genBestParams[sim], genScores[sim], randSeeds[sim], timeStamps[sim],\
        figGAProgresss[sim] = RunGARealOptim(parmsGA, parmsData, parmsObj, verb, randSeed)

# get the overall best
bestIndx = np.argmax(parmsGA['optimGoal']*np.array(bestScores))
bestScore = bestScores[bestIndx]
bestParam = bestParams[bestIndx]
timeStamp = timeStamps[bestIndx]